In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.lines import Line2D
import os
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE" # on NERSC filelocking is not allowed
import h5py
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Dense
import tensorflow.keras.backend as K
from sklearn.metrics import roc_curve, auc
import sklearn.metrics as sk
import pickle as pkl
import pandas as pd

import sys
# Path to dir model.py lives in -------
# NOTE: This needs to be modified to where your repo lives, path to /repo/path/VAE_FS/models/
# If the jupyter notebook kernel is running from VAE_FS/models/ the
# line below is not needed
sys.path.append('/global/homes/j/jananinf/projs/VAE_FS/models/')

# import the custom models and functions
from models import Qmake_encoder_set_weights, Qmake_decoder_set_weights, Qmake_discriminator, VAE_GAN_Model
from data_and_eval_utils import load_preprocessed_snl, plot_rocs, calc_anomaly_dist, AD_score_KL, AD_score_CKL, get_truth_and_scores, eval_rocs, SIG_KEYS
# from models import VAE_Model_ATLAS_beta as NNmodel


# # # Make notebook run on other GPUS. GPT's solution ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# gpus = tf.config.list_physical_devices('GPU')
# tf.config.set_visible_devices(gpus[0], 'GPU')  # change 1 to 0, 2, 3 as needed
# # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

2025-08-12 06:49:37.787776: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-12 06:49:41.635246: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from gan_params import *
print_base_params()

CONSTANTS IMPORTED:
            NUM_TRAIN      = 10 # Number of iterations to train for.
            # VAE Architecture
            INPUT_SZ       = 57
            H1_SZ          = 32 # Hidden layer 1 size
            H2_SZ          = 16 # "          " 2 "  "
            LATENT_SZ      = 3
            # Discriminator Architecture # 8, 2 is on ATLAS-VAE-GAN
            DISC_H1_SZ     = 8 # Size of first hidden layer of discriminator  
            DISC_H2_SZ     = 2 # "" second hidden layer ""
            # Training schedule and parameters
            NUM_EPOCHS     = 100
            STEPS_EPOCH    = 20 # Steps per epoch
            BATCH_SIZE     = 1024
            STOP_PATIENCE  = 40
            LR_PATIENCE    = 20
            LR             = 0.001 # Learning rate
            REDUCE_LR_FACTOR = 0.5
            VAL_SPLIT      = 0.2 # Validation split
            CYCLE_LEN      = 20
            SHUFFLE_BOOL   = True
            # Hyperparameters
            MIN_BETA       = 0
          

In [3]:
home_path = "/global/cfs/cdirs/m2616/jananinf/projsIO/VAE_FS/" # Updated to NERSC
SAVE_PATH = home_path+f"GAN_trainings/" 

### Loss plots.

In [4]:
# History Keys
keys = [
        'loss'               # VAE total loss term.
        ,'reco_loss'         # VAE loss term
        ,'kl_loss'           # VAE Loss term
        ,'disc_loss'         # VAE loss due to discriminator "failure to fool disc"
        # # ,'raw_loss'          # Reco_loss + kl_loss
        ,'w_kl_loss'         # kl_loss * beta
        ,'w_disc_loss'       # disc_loss * gamma
        ,'d_loss'
        # Validation version
        ,'val_loss'          
        ,'val_reco_loss'
        ,'val_kl_loss'
        ,'val_disc_loss'
        # ,'val_raw_loss'
        ,'val_w_kl_loss'
        ,'val_w_disc_loss'
        ,'val_d_loss'
        # --
        # ,'beta'              # hyperparameter
        # ,'gamma'             # hyperparameter
        # ,'val_gamma'         # hyperparameter
        # ,'val_beta'          # hyperparameter
        # ,'lr'              # learning rate
        ]

color_key = {
             'loss' : 'k'               # VAE total loss term.
            ,'val_loss' : 'k'         
            ,'reco_loss': 'tab:blue'         # VAE loss term
            ,'val_reco_loss' : 'tab:blue'
            ,'kl_loss': 'crimson'          # VAE Loss term
            ,'val_kl_loss': 'crimson'
            ,'disc_loss' : 'c'        # VAE loss due to discriminator "failure to fool disc"
            ,'val_disc_loss' : 'c'
            ,'w_kl_loss'  : 'tab:orange'        # kl_loss * beta
            ,'val_w_kl_loss' : 'tab:orange'
            ,'w_disc_loss'  : 'tab:green'     # disc_loss * gamma
            ,'val_w_disc_loss' : 'tab:green'
            ,'d_loss': 'r'
            ,'val_d_loss' :'r'
        }
# d_loss : discriminator loss
# loss : generator total loss
# raw_loss : reconstruction and kl_loss without beta weighting


# # Generate cleaner legend
# proxy_lines = {}
# for key in keys:
#     base_key = key.replace('val_', '')  # Strip 'val_' to group them
#     if base_key not in proxy_lines and key in color_key:
#         proxy_lines[base_key] = Line2D([0], [0], 
#                                     color=color_key[key], 
#                                     lw=2, 
#                                     label=base_key)
# clean_leg = list(proxy_lines.values())
# for att_n in range(6, 38): # plot all attempts. most recent is 18.
#     att_path = SAVE_PATH + f"attempt{att_n}/"

#     # Make folder for loss plots if it doesn't exist
#     plot_dir = os.path.join(SAVE_PATH, f"loss_plots/attempt{att_n}/")
#     os.makedirs(plot_dir, exist_ok=True)
    
#     for i in range(10): # Currently only training 10 models at a time.
#         save_path = att_path + f"n_{i}/"
#         with open(save_path + 'training_history.pkl', 'rb') as f:
#             history = pkl.load(f)
    
        
#         # Plot training losses
#         # fig, (ax, ax2) = plt.subplots(nrows=2, sharex=True, figsize=(8,10))
#         fig = plt.figure(figsize=(12, 8))
#         gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1])  # 3:1 means top gets 75%, bottom 25%

#         ax = fig.add_subplot(gs[0])
#         ax2 = fig.add_subplot(gs[1], sharex=ax)

#         # Calculate fractional contributions to total VAE loss
#         loss = np.array(history['loss'])
#         reco_loss = np.array(history['reco_loss'])
#         beta = np.array(history['beta'])
#         reco_loss_frac = (reco_loss * (1 - beta))/loss

#         w_kl_loss_frac = np.array(history['w_kl_loss'])/loss
#         w_disc_loss_frac = np.array(history['w_disc_loss'])/loss
#         ax2.plot(reco_loss_frac, label='reco_loss_frac')
#         ax2.plot(w_kl_loss_frac, label='w_kl_loss_frac')
#         ax2.plot(w_disc_loss_frac, label='w_disc_loss_frac')

#         # Tweak fractional plot
#         # ax2.set_ylim((0,1))
#         ax2.set_ylabel('Approximate\nTotal VAE Loss fraction')
#         # ax2.tick_params(axis='y', labelcolor='b')
#         ax2.legend()
#         ax2.grid()
#         ax2.set_xlabel('Epoch')

#         for key in keys:
#             if key == 'lr' or history.get(key) == None:
#                 continue
#             ax.plot(np.abs(history[key]),
#                      label=key, 
#                      linestyle = "dashed" if key[0:3] == 'val' else "solid",
#                      marker= "x" if key[0:3] == 'val' else "o",
#                      markersize=6.5,
#                      color=color_key[key])
    
#         # Customize the plot
#         ax.set_title(f'Training and Validation Losses, Attempt: {att_n} Run: {i}')
#         ax.set_ylabel('Loss')
#         # ax.tick_params(axis='y', labelcolor='r')
#         # ax.legend()
#         ax.grid(True)
#         ax.set_yscale('log')
#         ax.legend(handles=clean_leg, title="○ = train, x = val")
#         plt.savefig(SAVE_PATH + f"loss_plots/attempt{att_n}/" + f"loss_attempt_{att_n}_run_{i}.png", bbox_inches='tight')
#         # plt.show()
#         plt.close(fig)
#     print(f"Attempt {att_n} plotting complete!")

In [5]:
# Load data
data = load_preprocessed_snl()
# X_train = data['X_train']

Data loaded from preprocessed_SNL_data.h5


##### Calculate Anomaly scores

After inspecting the graphs a few notable models remain

In [6]:
 # mins 88, 90, 89, 79 for AUC. 
# # 16 did the best AUC and I think also has higher TPR @ target FPR

Generate ROC Curves for all trained iterations of the model. Save them and 
generate a list of of models with their AUC to rank them later

In [7]:
# Load the model
new_enc = Qmake_encoder_set_weights(INPUT_SZ, H1_SZ, H2_SZ, LATENT_SZ)
new_dec = Qmake_decoder_set_weights(INPUT_SZ, H1_SZ, H2_SZ, LATENT_SZ)
new_disc = Qmake_discriminator(INPUT_SZ, DISC_H1_SZ, DISC_H2_SZ)
new_VAE = VAE_GAN_Model(new_enc, new_dec, new_disc)
opt = keras.optimizers.Adam(learning_rate=LR) # These help silence benign warnings and is cleaner ----
new_VAE.compile(optimizer=opt)                # ---


FIG_SAVE_PATH = SAVE_PATH + "roc_plots/"

# for att_n in range(6,17): # Splitting everythingup
#     roc_results = {}
#     bad_iters = {}

#     # Iterate through its iterations
#     fig_save_path = FIG_SAVE_PATH + f"attempt{att_n}/"

#     for i in range(NUM_TRAIN):
#         save_path = SAVE_PATH + f"attempt{att_n}/n_{i}/"

#         new_VAE.load_weights(save_path)
#         just_enc = new_VAE.get_layer("encoder") # We only need encoder output

#         roc_perfs = eval_rocs(just_enc, data, AD_score_CKL)

#         if roc_perfs is None:
#             print(f"Bad Iteration. Attempt: {att_n}, Iteration: {i}.")
#             bad_iters.setdefault(f"Attempt{att_n}", []).append(i) # Create the entry and list if it doesn't exit, otherwise append it to the current list of that entry
#             continue
#         # Save its auc performance
#         roc_results[f"Attempt{att_n}_iter_{i}"] = {k: roc_perfs[k]['auc'] for k in SIG_KEYS.keys()}

#         # Commented out because we don't need to make the plots rn
#         # Make folder for roc plots if it doesn't exist
#         # plot_dir = os.path.join(fig_save_path)
#         # os.makedirs(plot_dir, exist_ok=True)
#         # f = plot_rocs(roc_perfs, f"ROC Curves. CKL as Anomaly Score. Attempt: {att_n}, Iter: {i}")
#         # f.savefig(fig_save_path + f"roc_att{att_n}_iter{i}.png", bbox_inches = 'tight')
#         # plt.close(f)
#         # print(f"Roc curve plotted and saved. for Attempt: {att_n}, iter: {i}!")

#         print(f"Completed Attempt: {att_n} Iter: {i}")
        
#     with open(SAVE_PATH + f"temp_pkls/roc_results_att_{att_n}.pkl", "wb") as f:
#         pkl.dump([bad_iters, roc_results], f)

2025-08-12 06:50:08.118889: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 890 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:c3:00.0, compute capability: 8.0


In [40]:
all_results = pd.DataFrame({})
stats = pd.DataFrame({})

for i in range(6, 50):
    with open(SAVE_PATH + f"temp_pkls/roc_results_att_{i}.pkl", "rb") as f:
        roc_result = pkl.load(f)[1] # Load just the roc results
    temp = pd.DataFrame(roc_result).transpose()
    all_results = pd.concat([all_results, temp], axis=0)
    # Some quick stats while we're at it.
    temp_stats = {}
    for c in temp.columns:
        temp_stats[c] = temp[c].mean()
    temp_stats = pd.DataFrame(temp_stats, index = [f"attempt_{i}"])
    # temp_stats["RMS_AUC"] = np.sqrt(temp.Ato4l**2 + temp.hToTauTau**2 + temp.hChToTauNu**2 + temp.leptoquark**2)
    stats = pd.concat([stats, temp_stats], axis = 0)

In [46]:
stats["RMS_AUC"] = np.sqrt((stats.Ato4l**2 + stats.hToTauTau**2 + stats.hChToTauNu**2 + stats.leptoquark**2)/4)
stats["MEAN_AUC"] = stats.iloc[:, :4].mean(axis=1)
stats["SPREAD"] = stats.iloc[:, :4].max(axis=1) - stats.iloc[:, :4].min(axis=1)

In [47]:
stats = stats.sort_values(by='RMS_AUC', ascending=False)
stats

,Ato4l,hToTauTau,hChToTauNu,leptoquark,RMS_AUC,MEAN_AUC,SPREAD
attempt_47,0.911347,0.759736,0.859302,0.841065,0.844622,0.842862,0.151611
attempt_48,0.909307,0.755611,0.857954,0.838085,0.842060,0.840239,0.153696
attempt_44,0.904997,0.749910,0.857920,0.836969,0.839334,0.837449,0.155088
attempt_20,0.900845,0.753955,0.850843,0.831969,0.836074,0.834403,0.146890
attempt_39,0.899647,0.744628,0.854362,0.828230,0.833627,0.831717,0.155019
attempt_6,0.898694,0.750365,0.841409,0.822481,0.829931,0.828237,0.148330
attempt_29,0.895840,0.740885,0.846353,0.827478,0.829529,0.827639,0.154955
attempt_43,0.897580,0.745341,0.842612,0.822376,0.828776,0.826977,0.152239
attempt_40,0.893677,0.740226,0.845324,0.824270,0.827736,0.825874,0.153451
attempt_26,0.892208,0.744810,0.843951,0.822187,0.827500,0.825789,0.147398


In [27]:
np.sqrt(np.sum([1 for _ in range(3)]))

1.7320508075688772

In [51]:
all_results["RMS_AUC"] = np.sqrt((all_results.Ato4l**2 + all_results.hToTauTau**2 + all_results.hChToTauNu**2 + all_results.leptoquark**2)/4)
all_results["MEAN_AUC"] = all_results.iloc[:, :4].mean(axis=1)
all_results["SPREAD"] = all_results.iloc[:, :4].max(axis=1) - all_results.iloc[:, :4].min(axis=1)

In [52]:
# Drop the row if it has any AUC less than 0.79. 
threshold = 0.79
cols = ["Ato4l", "hToTauTau", "hChToTauNu", "leptoquark"]
all_results = all_results.loc[(all_results[cols] >= threshold).all(axis=1)]

In [53]:
all_results = all_results.sort_values(by='RMS_AUC', ascending=False)
all_results

,Ato4l,hToTauTau,hChToTauNu,leptoquark,RMS_AUC,MEAN_AUC,SPREAD
Attempt30_iter_3,0.904281,0.795604,0.931567,0.900733,0.884569,0.883046,0.135963
Attempt20_iter_7,0.924522,0.799560,0.898655,0.881706,0.877358,0.876111,0.124962
Attempt26_iter_1,0.909648,0.806666,0.900492,0.888728,0.877339,0.876384,0.102982
Attempt9_iter_6,0.922363,0.799251,0.905183,0.874381,0.876563,0.875294,0.123113
Attempt37_iter_3,0.925020,0.795849,0.895894,0.880595,0.875659,0.874340,0.129172
Attempt43_iter_6,0.922287,0.791996,0.891548,0.874944,0.871530,0.870194,0.130291
Attempt43_iter_0,0.921156,0.790336,0.891824,0.873129,0.870469,0.869111,0.130819
Attempt35_iter_1,0.919565,0.793042,0.884439,0.875767,0.869442,0.868203,0.126523
Attempt19_iter_0,0.922172,0.799977,0.880096,0.867896,0.868645,0.867535,0.122195
Attempt18_iter_9,0.917649,0.795939,0.883224,0.868417,0.867443,0.866307,0.121711


In [27]:
all_results = all_results.sort_values(by="MEAN_AUC", ascending = False)
all_results

,Ato4l,hToTauTau,hChToTauNu,leptoquark,RMS_AUC,MEAN_AUC,SPREAD
Attempt30_iter_3,0.904281,0.795604,0.931567,0.900733,1.769137,0.883046,0.135963
Attempt26_iter_1,0.909648,0.806666,0.900492,0.888728,1.754678,0.876384,0.102982
Attempt20_iter_7,0.924522,0.799560,0.898655,0.881706,1.754715,0.876111,0.124962
Attempt9_iter_6,0.922363,0.799251,0.905183,0.874381,1.753127,0.875294,0.123113
Attempt37_iter_3,0.925020,0.795849,0.895894,0.880595,1.751317,0.874340,0.129172
Attempt43_iter_6,0.922287,0.791996,0.891548,0.874944,1.743059,0.870194,0.130291
Attempt43_iter_0,0.921156,0.790336,0.891824,0.873129,1.740938,0.869111,0.130819
Attempt35_iter_1,0.919565,0.793042,0.884439,0.875767,1.738884,0.868203,0.126523
Attempt19_iter_0,0.922172,0.799977,0.880096,0.867896,1.737290,0.867535,0.122195
Attempt18_iter_9,0.917649,0.795939,0.883224,0.868417,1.734887,0.866307,0.121711


In [20]:
all_results = all_results.sort_values(by='SPREAD')
all_results

,Ato4l,hToTauTau,hChToTauNu,leptoquark,RMS_AUC,MEAN_AUC,SPREAD
Attempt8_iter_7,0.810077,0.730131,0.800311,0.745802,1.544680,0.771580,0.079947
Attempt14_iter_2,0.719395,0.729189,0.806499,0.776158,1.517270,0.757810,0.087104
Attempt40_iter_7,0.885304,0.792607,0.864640,0.862436,1.703933,0.851247,0.092697
Attempt32_iter_3,0.850447,0.754521,0.845967,0.821162,1.637843,0.818024,0.095926
Attempt8_iter_5,0.883276,0.780738,0.880568,0.848191,1.698398,0.848193,0.102538
...,...,...,...,...,...,...,...
Attempt37_iter_8,0.829493,0.571376,0.684756,0.667095,1.388681,0.688180,0.258117
Attempt46_iter_3,0.757566,0.493914,0.580576,0.507523,1.188488,0.584895,0.263651
Attempt34_iter_3,0.756485,0.478363,0.491274,0.494872,1.134614,0.555249,0.278121
Attempt35_iter_0,0.684154,0.414690,0.430752,0.403990,0.994379,0.483396,0.280164


In [24]:
all_results.loc[:4]

TypeError: cannot do slice indexing on Index with these indexers [4] of type int

In [ ]:

# data = [leptoquark_data, Ato4l_data, hChToTauNu_data, hToTauTau_data
#                , X_train
#                , X_test
#                ] # Already defined.
data_names_tex = [ # latex version
                "Leptoquark"
                , "$A\\rightarrow 4\ell$"
                , "$h^{\pm}\\rightarrow\\tau \\nu$"
                , "$h^0\\rightarrow\\tau\\tau$"
                , "Training Set (BG)" # Background
                , "Test Set (BG)" # Background
                ]

anomaly_scores = []
for _, dat in data.items():
    s = calc_anomaly_dist(dat, just_enc, AD_score_CKL)
    anomaly_scores.append(s)


In [ ]:
# plot setting for CKL
bin_n = 125
xlims = (0, 40)
ylims = (0, 0.03)
bins  = np.linspace(xlims[0], xlims[1], bin_n)
xlabel = "Clipped KL"


# # Investigating around the threshold at 161
# ckl_roc_threshold = 161.84
# bin_n = 10
# l_margin = 10 
# r_margin = 300

# xlims = ( ckl_roc_threshold - l_margin , ckl_roc_threshold + r_margin)
# ylims = (0, 0.01)
# bins  = np.linspace(xlims[0], xlims[1], bin_n)
# xlabel = "Clipped KL"

# Plot settings for KL
# bin_n = 125
# xlims = (0, 40)
# ylims = (0, 0.0125)
# bins  = np.linspace(0, xlims[1], bin_n)
# xlabel = "KL Divergence"

for i in range(len(data_names_tex)):
    dat = anomaly_scores[i]
    # print(bin_n)
    plt.hist(dat
             , bins = bins
             , label=data_names_tex[i] # + " " + str(bin_n)
             , histtype = "step"
             , density=True
             )
plt.legend(loc="upper right")
# plt.vlines(ckl_roc_threshold, 0, 1)
# plt.loglog()
# plt.semilogy()
# plt.semilogx()
plt.xlabel(xlabel)
plt.ylabel("Density")
plt.grid()
plt.ylim(ylims)
plt.xlim(xlims)
plt.title("Anomaly Score Distribution Across Datasets")